In [1]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import os
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import json

In [2]:
input_folder_path = "New files"
output_folder_path = "ParsedJson"

In [3]:
# Connection to the document recognizer we trained on Azure
with open ('Config.json', 'r') as json_file:
    data = json.load(json_file)
    endpoint = data["endpoint"]
    key = data["key"]

model_id = "Annual-Tonnage-Feb16-Signature"
document_analysis_client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))

In [ ]:
for filename in os.listdir(input_folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(input_folder_path, filename)
        with open(file_path, "rb") as document:
            print(f"-------->Parsing {filename}<---------")
            content = document.read()
            try:
                poller = document_analysis_client.begin_analyze_document(model_id=model_id, document=content)
                result = poller.result()
                data = result.to_dict()
                output_path = os.path.join(output_folder_path, f"{os.path.splitext(filename)[0]}")
                with open(output_path + '.json', 'w') as json_file:
                    json.dump(data, json_file, indent=4)
            except Exception as e:
                print(f"Error processing file {filename}: {str(e)}")
                continue

## Folder Json

### Section A

In [4]:
data_list_A = []

for filename in os.listdir(output_folder_path):
    if filename.endswith(".json"):
        file = os.path.join(output_folder_path, filename)

        with open(file, "r") as json_file:
            print(f"-------->Reading {filename}<---------")
            json_data = json.load(json_file)

            style_list = json_data.get("styles", [])
            handwritten = {}  
            for style in style_list:
                if style.get("is_handwritten", False):  
                    handwritten["Handwritten?"] = "True"
                else:
                    handwritten["Handwritten?"] = "False"

            documents_list = json_data.get("documents", [])
            for document in documents_list:
                field_data_A = {"Filename": filename}
                for name, field in document.get("fields", {}).items():
                    field_value_A = field.get("valueString", "") if field.get("valueString", "") else field.get("content", "")
                    field_data_A[name] = field_value_A
                field_data_A.update(handwritten) 
                data_list_A.append(field_data_A)

headers_A = [
    'Filename',
    'Company',
    'Contact Person',
    'Title',
    'Address',
    'City',
    'State',
    'Zip Code',
    'Phone',
    'Fax',
    'Email',
    'Paper Collection',
    'Electronics Recycling',
    'Secure Document Destruction',
    'Tire Recycling',
    'Solvents/Oil/Antifreeze Recycling',
    'Metals Recycling (Ferrous & Non-Ferrous)',
    'Battery Recycling',
    'Yard Maintenance/Landscaping',
    'Textile, Shoes, and/or Clothing Recycling',
    'Commercial Back-Hauling for Cardboard and Other',
    'Materials Collection (grocery and other retail outlets)',
    'Solid Waste Collection (trash and recycling)',
    'Construction and Demolition Debris Collection',
    'Bulk Waste & Junk Collection',
    'Material Recovery Facility (MRF)',
    'Other',
    'Other-Text',
    'Printed Signature',
    'Signed Date',
    'Methodology',
    'Comments',
    'Handwritten?'
]

df_A = pd.DataFrame(data_list_A, columns=headers_A)
df_A['Phone'] = df_A['Phone'].str.replace(r'[^0-9]', '', regex=True)
df_A['Fax'] = df_A['Fax'].str.replace(r'[^0-9]', '', regex=True)
df_A['City'] = df_A['City'].str.replace(r'[^a-zA-Z]', '', regex=True)
df_A.replace('', "NULL", inplace=True)
df_A.replace(pd.NA, "NULL", inplace=True)

df_A.head(50)


-------->Reading 2023 Annual Tonnage Report ACE.json<---------
-------->Reading 2023 Annual Tonnage Report Chapel Valley.json<---------
-------->Reading 2023 Annual Tonnage Report Zero Waste Solutions.json<---------
-------->Reading 2023 Arlington Annual Tonnage Report-United Rentals.json<---------
-------->Reading 2023 Arlington Annual Tonnage Report.json<---------
-------->Reading 2023 Glenayr Apartments Tonnage Report.json<---------
-------->Reading 2023 Tonnage Report - Bartlett Tree.json<---------
-------->Reading ADS 2023 Tonnage Report.json<---------
-------->Reading Atlantic Biofuels Tonnage Report.json<---------
-------->Reading Capitol Services Tonnage Report 2024 Arlington County.json<---------
-------->Reading Compost Crew Annual Tonnage Report Arlingon.json<---------
-------->Reading Concrete Protection and Restoration 2023 Tonnage Report.json<---------
-------->Reading Darling Ingredients Tonnage Report.json<---------
-------->Reading Dittmar 2023 Annual Tonnage Report.js

,Filename,Company,Contact Person,Title,Address,City,State,Zip Code,Phone,Fax,Email,Paper Collection,Electronics Recycling,Secure Document Destruction,Tire Recycling,Solvents/Oil/Antifreeze Recycling,Metals Recycling (Ferrous & Non-Ferrous),Battery Recycling,Yard Maintenance/Landscaping,"Textile, Shoes, and/or Clothing Recycling",Commercial Back-Hauling for Cardboard and Other,Materials Collection (grocery and other retail outlets),Solid Waste Collection (trash and recycling),Construction and Demolition Debris Collection,Bulk Waste & Junk Collection,Material Recovery Facility (MRF),Other,Other-Text,Printed Signature,Signed Date,Methodology,Comments,Handwritten?
0,2023 Annual Tonnage Report ACE.json,Ace Deconstruction LLC,Anchan Ladia,CFO,13804 Dawson Beach Road,Woodbridge,VA,22191,NULL,NULL,anchan@acedeconstruction.com,:selected:,:unselected:,:unselected:,:unselected:,:unselected:,:selected:,:selected:,:unselected:,:unselected:,:selected:,:unselected:,:selected:,:selected:,:unselected:,:unselected:,:unselected:,NULL,Anchan Ladia,1/09/2024,NULL,NULL,True
1,2023 Annual Tonnage Report Chapel Valley.json,Chapel Valley Landscape Company,JIll Hartkopf,Branch Manager,3041 Colvin Street,Alexandria,VA,22314,7035985514,NULL,jillh@chapelvalley.com,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:selected:,:unselected:,:unselected:,:unselected:,:unselected:,NULL,Jill Hartkopf,01/05/2024,NULL,NULL,True
2,2023 Annual Tonnage Report Zero Waste Solution...,"ZERO WASTE SOLUTIONS, INC.",JESSICA SEXTON,ACCOUNTS PAYABLE SPECIALIST,"PO BOX 5097,",CONCORD,CA,94524,9252703339,NULL,ACCOUNTING @ZEROWASTESOLUTIONS.COM,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:selected:,:selected:,:unselected:,:unselected:,:unselected:,NULL,JESSICA SEXTON,1/31/2024,NULL,NULL,True
3,2023 Arlington Annual Tonnage Report-United Re...,"United Rentals (North America), Inc .; DBA Rel...",Jeffrey Walker,Environmental Manager,"11825 Livingston Rd,",Manassas,VAS,20109,7036403901,NULL,environmental@ur.com; jwalker5@ur.com,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:selected:,Septage/Portable Toilet Waste,Jeffrey Walker,1/3/2024,"We estimate we will collect 200,000 gallons of...",NULL,True
4,2023 Arlington Annual Tonnage Report.json,Brandon Rushing Lawn and Garden Care,Tamela Addison,Support Team Manager,"7805 Cinder Bed Road,",NULL,NULL,NULL,7033390067,7036461009,tammy@brandonrushing.com,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:selected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,NULL,NULL,NULL,1 bag of debris per client for 40 weeks per ye...,NULL,NULL
5,2023 Glenayr Apartments Tonnage Report.json,Glenayr Apartments LLC,Arthur E House,Owner,"399 N. Park Drive Arlington,",NULL,VA,22203,7035227354,7035243805,glenayrapt@aol.com,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:selected:,:unselected:,:unselected:,NULL,Deborah J. House,1/3/2024,NULL,Appliance and Furniture,True
6,2023 Tonnage Report - Bartlett Tree.json,BARTLETT TREEE EXPERTS,MARGOT ARELLANO,LEAD ADMIN ASST,"7823 LOISDALE RD, STE LL,",SPRINGFIELD,VA,22150,7035506900,70315502656,MARELLANO@BARTLETT.COM,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:selected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,:unselected:,NULL,NULL,12/28/25,Remington Mulch does not provide tonnage or me...,same,True
7,ADS 2023 Tonnage Report.json,American Disposal Services Inc.,Kyle B

### Section B

In [5]:
def code_strings(df, column_name):
  mapping = {'Yes': 'Y', 'yes': 'Y', 'YES': 'Y', 'No': 'N', 'no': 'N', 'NO': 'N'}
  df[column_name] = df[column_name].replace(mapping)
  return df

In [6]:
SectionBTotal = pd.DataFrame()
SkippedFiles = []
for filename in os.listdir(output_folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(output_folder_path, filename)
        try:
            with open(file_path, "r") as json_file:
                print(f"-------->Reading {filename}<---------")

                json_data = json.load(json_file)
                documents_list = json_data.get("documents", [])
                for document in documents_list:
                    SectionBT = pd.DataFrame()
                    
                    fields_dict = document.get("fields", {})
                    section_b_page1 = fields_dict.get("SectionB-Page1", {})
                    section_b_page2 = fields_dict.get("SectionB-Page2", {})
                    value_list = section_b_page1.get("value", [])
                    value_list2 = section_b_page2.get("value", [])
                    data_list_B = []
                    data_list_B2 = []

                    for item in value_list:
                        field_data_B = {}
                        column1_value = item.get("value", {}).get("COLUMN1", {}).get("value", "")
                        field_data_B["Material"] = column1_value

                        tons_recycled_value = item.get("value", {}).get("Tons Recycled", {}).get("value", "")
                        field_data_B["Tons Recycled"] = tons_recycled_value

                        estimate_value = item.get("value", {}).get("Estimate? (Y/N)", {}).get("value", "")
                        field_data_B["Estimate? (Y/N)"] = estimate_value

                        source_code_value = item.get("value", {}).get("Source Code", {}).get("value", "")
                        field_data_B["Source Code"] = source_code_value

                        facility_value = item.get("value", {}).get("Name & Address of Processing Facility(s) by ton", {}).get("value", "")
                        field_data_B["Name & Address of Processing Facility(s) by ton"] = facility_value
                        
                        data_list_B.append(field_data_B)
                    df_B = pd.DataFrame(data_list_B)

                    for item in value_list2:
                        field_data_B2 = {}
                        column1_value = item.get("value", {}).get("COLUMN1", {}).get("value", "")
                        field_data_B2["Material"] = column1_value

                        tons_recycled_value = item.get("value", {}).get("Tons Recycled", {}).get("value", "")
                        field_data_B2["Tons Recycled"] = tons_recycled_value

                        estimate_value = item.get("value", {}).get("Estimate? (Y/N)", {}).get("value", "")
                        field_data_B2["Estimate? (Y/N)"] = estimate_value

                        source_code_value = item.get("value", {}).get("Source Code", {}).get("value", "")
                        field_data_B2["Source Code"] = source_code_value

                        facility_value = item.get("value", {}).get("Name & Address of Processing Facility(s) by ton", {}).get("value", "")
                        field_data_B2["Name & Address of Processing Facility(s) by ton"] = facility_value

                        data_list_B2.append(field_data_B2)
                    df_B_2 = pd.DataFrame(data_list_B2)
                    
                    SectionB = pd.concat([df_B, df_B_2], axis=0)

                    SectionB.reset_index(drop=True, inplace=True)
                    # Removing Unwanted characters
                    SectionB.iloc[:, 1] = SectionB.iloc[:, 1].replace(',', '', regex=True)
                    SectionB = SectionB.map(lambda x: str(x).replace(':selected:', ''))
                    SectionB = SectionB.map(lambda x: str(x).replace('\n', ''))
                    SectionB = SectionB.map(lambda x: str(x).replace('nan', ''))

                    strings_to_check = ['TONS RECYCLED', 'ESTIMATE? (Y/N)', 'SOURCE CODE', 'NAME & ADDRESS OF PROCESSING FACILITY(S) BY TON']
                    SectionB = SectionB[~(SectionB.iloc[:, 1].str.upper().isin(strings_to_check))]# | SectionB.iloc[:, 2].str.upper().isin(strings_to_check))]
                    SectionB.iloc[:, 0] = SectionB.iloc[:, 0].replace("", None).ffill()

                    # Adding columns
                    SectionB['Filename'] = filename
                    SectionB['Section'] = 'Section B'
                    
                    SectionBTotal = pd.concat([SectionBTotal, SectionB], ignore_index=True)
        except Exception as e:
            SkippedFiles.append(filename)
            print(f"Skipping file {filename} due to error: {str(e)}")
            continue


# applying string transofrmation fr Source Code
SectionBTotal = code_strings(SectionBTotal, 'Estimate? (Y/N)')

# Removing Materials with Null values
SectionBTotal['Tons Recycled'] = SectionBTotal['Tons Recycled'].str.replace(r'[^\d. -]', '', regex=True)
SectionBTotal = SectionBTotal[SectionBTotal['Tons Recycled'].isin([0, "0", "o", pd.NA, 'N/A']) == False]
SectionBTotal = SectionBTotal.replace("", "NULL")
#SectionBTotal = SectionBTotal.dropna(subset=['Tons Recycled'])

SectionBTotal.head(10)

-------->Reading 2023 Annual Tonnage Report ACE.json<---------
-------->Reading 2023 Annual Tonnage Report Chapel Valley.json<---------
-------->Reading 2023 Annual Tonnage Report Zero Waste Solutions.json<---------
-------->Reading 2023 Arlington Annual Tonnage Report-United Rentals.json<---------
-------->Reading 2023 Arlington Annual Tonnage Report.json<---------
-------->Reading 2023 Glenayr Apartments Tonnage Report.json<---------
-------->Reading 2023 Tonnage Report - Bartlett Tree.json<---------
-------->Reading ADS 2023 Tonnage Report.json<---------
-------->Reading Atlantic Biofuels Tonnage Report.json<---------
-------->Reading Capitol Services Tonnage Report 2024 Arlington County.json<---------
-------->Reading Compost Crew Annual Tonnage Report Arlingon.json<---------
-------->Reading Concrete Protection and Restoration 2023 Tonnage Report.json<---------
-------->Reading Darling Ingredients Tonnage Report.json<---------
-------->Reading Dittmar 2023 Annual Tonnage Report.js

,Material,Tons Recycled,Estimate? (Y/N),Source Code,Name & Address of Processing Facility(s) by ton,Filename,Section
0,Single Stream/All in One Recycling,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section B
1,Bottles & Cans Only (no paper items),NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section B
2,Mixed Paper,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section B
3,Cardboard,104,N,NULL,Broad Run,2023 Annual Tonnage Report ACE.json,Section B
4,Newspapers,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section B
5,Shredded Paper,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section B
6,Appliances,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section B
7,Inoperative Motor Vehicles,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section B
8,Aluminum Cans,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section B
9,Other Scrap Metal,59,N,NULL,Woodbridge Metal,2023 Annual Tonnage Report ACE.json,Section B


In [7]:
word_list_data = []

for filename in os.listdir(output_folder_path):
    if filename.endswith(".json"):
        file = os.path.join(output_folder_path, filename)
        with open(file, 'r') as json_file:
            json_data = json.load(json_file)
            page_list = json_data.get("pages", [])
            for page in page_list:
                word_list = page.get("words", [])
                for word in word_list:
                    word_content = word["content"]
                    word_confidence = word["confidence"]
                    word_list_data.append({"word": word_content, "confidence": word_confidence, "filename": filename})

word_df = pd.DataFrame(word_list_data)
word_df.head(10)


,word,confidence,filename
0,ARLINGTON,0.993,2023 Annual Tonnage Report ACE.json
1,VIRGINIA,0.992,2023 Annual Tonnage Report ACE.json
2,Page,0.993,2023 Annual Tonnage Report ACE.json
3,1,0.997,2023 Annual Tonnage Report ACE.json
4,of,0.995,2023 Annual Tonnage Report ACE.json
5,9,0.995,2023 Annual Tonnage Report ACE.json
6,Arlington,0.993,2023 Annual Tonnage Report ACE.json
7,County,0.994,2023 Annual Tonnage Report ACE.json
8,Recycling,0.993,2023 Annual Tonnage Report ACE.json
9,&,0.964,2023 Annual Tonnage Report ACE.json


In [8]:
def calculate_average_confidence(material, filename):
    confidences = []
    current_word = ""
    for word in material.split():
        current_word += word
        confidence = word_df[(word_df['word'] == current_word) & (word_df['filename'] == filename)]['confidence'].values
        if len(confidence) > 0:
            confidences.append(confidence[0])
            current_word = ""
    return sum(confidences) / len(confidences) if confidences else 0

text_columns = [col for col in SectionBTotal.columns if col not in ['Filename', 'Section']]
SectionBTotal_Confidence = SectionBTotal.copy()
for col in text_columns:
    SectionBTotal_Confidence[col + '_confidence'] = SectionBTotal_Confidence.apply(lambda row: calculate_average_confidence(str(row[col]), row['Filename']), axis=1)

# Replacing zeros with hundred
SectionBTotal_Confidence.replace(0, "NULL", inplace=True)

SectionBTotal_Confidence.head()

,Material,Tons Recycled,Estimate? (Y/N),Source Code,Name & Address of Processing Facility(s) by ton,Filename,Section,Material_confidence,Tons Recycled_confidence,Estimate? (Y/N)_confidence,Source Code_confidence,Name & Address of Processing Facility(s) by ton_confidence
0,Single Stream/All in One Recycling,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section B,0.994600,NULL,NULL,NULL,NULL
1,Bottles & Cans Only (no paper items),NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section B,0.988571,NULL,NULL,NULL,NULL
2,Mixed Paper,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section B,0.995000,NULL,NULL,NULL,NULL
3,Cardboard,104,N,NULL,Broad Run,2023 Annual Tonnage Report ACE.json,Section B,0.995000,0.997,0.98,NULL,0.9945
4,Newspapers,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section B,0.994000,NULL,NULL,NULL,NULL


In [9]:
SectionCTotal = pd.DataFrame()

for filename in os.listdir(output_folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(output_folder_path, filename)
        try:
            with open(file_path, "r") as json_file:
                print(f"-------->Reading {filename}<---------")

                json_data = json.load(json_file)
                documents_list = json_data.get("documents", [])
                for document in documents_list:
                    SectionCT = pd.DataFrame()
                    
                    fields_dict = document.get("fields", {})
                    section_c = fields_dict.get("SectionC", {})
                    value_list = section_c.get("value", [])
                    
                    data_list_C = []

                    for item in value_list:
                        field_data_C = {}
                        column1_value = item.get("value", {}).get("COLUMN1", {}).get("value", "")
                        field_data_C["Material"] = column1_value

                        # Assuming these keys are present in your JSON structure
                        tons_recycled_value = item.get("value", {}).get("Tons Recycled", {}).get("value", "")
                        field_data_C["Tons Recycled"] = tons_recycled_value

                        estimate_value = item.get("value", {}).get("Estimate? (Y/N)", {}).get("value", "")
                        field_data_C["Estimate? (Y/N)"] = estimate_value

                        source_code_value = item.get("value", {}).get("Source Code", {}).get("value", "")
                        field_data_C["Source Code"] = source_code_value

                        facility_value = item.get("value", {}).get("Name & Address of Processing Facility(s) by ton", {}).get("value", "")
                        field_data_C["Name & Address of Processing Facility(s) by ton"] = facility_value

                        data_list_C.append(field_data_C)
                    SectionC = pd.DataFrame(data_list_C)
                    
                    SectionC.reset_index(drop=True, inplace=True)
                    SectionC.iloc[:, 1] = SectionC.iloc[:, 1].replace(',', '', regex=True)
                    SectionC = SectionC.map(lambda x: str(x).replace(':selected:', ''))
                    SectionC = SectionC.map(lambda x: str(x).replace('\n', ''))
                    SectionC = SectionC.map(lambda x: str(x).replace('nan', ''))

                    strings_to_check = ['TONS RECYCLED', 'ESTIMATE? (Y/N)', 'SOURCE CODE', 'NAME & ADDRESS OF PROCESSING FACILITY(S) BY TON']
                    SectionC = SectionC[~(SectionC.iloc[:, 1].str.upper().isin(strings_to_check))]# | SectionB.iloc[:, 2].str.upper().isin(strings_to_check))]
                    SectionC.iloc[:, 0] = SectionC.iloc[:, 0].replace("", None).ffill()

                    # Adding columns
                    SectionC['Filename'] = filename
                    SectionC['Section'] = 'Section C'

                    SectionCTotal = pd.concat([SectionCTotal, SectionC], ignore_index=True)

        except Exception as e:
            SkippedFiles.append(filename)
            print(f"Skipping file {filename} due to error: {str(e)}")
            continue

SectionCTotal = code_strings(SectionCTotal, 'Estimate? (Y/N)')
                        
# Removing Materials with Null values
SectionCTotal['Tons Recycled'] = SectionCTotal['Tons Recycled'].str.replace(r'[^\d. -]', '', regex=True)
SectionCTotal = SectionCTotal[SectionCTotal['Tons Recycled'].isin([0, "0", "o", pd.NA, 'N/A']) == False]
SectionCTotal = SectionCTotal.replace("", "NULL")
#SectionCTotal = SectionCTotal.dropna(subset=['Tons Recycled'])

SectionCTotal.head(10)

-------->Reading 2023 Annual Tonnage Report ACE.json<---------
-------->Reading 2023 Annual Tonnage Report Chapel Valley.json<---------
-------->Reading 2023 Annual Tonnage Report Zero Waste Solutions.json<---------
-------->Reading 2023 Arlington Annual Tonnage Report-United Rentals.json<---------
-------->Reading 2023 Arlington Annual Tonnage Report.json<---------
-------->Reading 2023 Glenayr Apartments Tonnage Report.json<---------
-------->Reading 2023 Tonnage Report - Bartlett Tree.json<---------
-------->Reading ADS 2023 Tonnage Report.json<---------
-------->Reading Atlantic Biofuels Tonnage Report.json<---------
-------->Reading Capitol Services Tonnage Report 2024 Arlington County.json<---------
-------->Reading Compost Crew Annual Tonnage Report Arlingon.json<---------
-------->Reading Concrete Protection and Restoration 2023 Tonnage Report.json<---------
-------->Reading Darling Ingredients Tonnage Report.json<---------
-------->Reading Dittmar 2023 Annual Tonnage Report.js

,Material,Tons Recycled,Estimate? (Y/N),Source Code,Name & Address of Processing Facility(s) by ton,Filename,Section
0,Fluorescent Lamps,1.5,N,NULL,Lighting Resources,2023 Annual Tonnage Report ACE.json,Section C
1,Paint,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section C
2,"Textiles (clothing, shoes, leather, etc.)",NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section C
3,Furniture,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section C
4,Books,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section C
5,Fluorescent Lamps,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report Chapel Valley.json,Section C
6,Paint,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report Chapel Valley.json,Section C
7,"Textiles (clothing, shoes, leather, etc.)",NULL,NULL,NULL,NULL,2023 Annual Tonnage Report Chapel Valley.json,Section C
8,Furniture,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report Chapel Valley.json,Section C
9,Books,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report Chapel Valley.json,Section C


In [10]:
text_columns = [col for col in SectionCTotal.columns if col not in ['Filename', 'Section']]
SectionCTotal_Confidence = SectionCTotal.copy()
for col in text_columns:
    SectionCTotal_Confidence[col + '_confidence'] = SectionCTotal_Confidence.apply(lambda row: calculate_average_confidence(str(row[col]), row['Filename']), axis=1)

# Replacing zeros with hundred
SectionCTotal_Confidence.replace(0, "NULL", inplace=True)

SectionCTotal_Confidence.head()

,Material,Tons Recycled,Estimate? (Y/N),Source Code,Name & Address of Processing Facility(s) by ton,Filename,Section,Material_confidence,Tons Recycled_confidence,Estimate? (Y/N)_confidence,Source Code_confidence,Name & Address of Processing Facility(s) by ton_confidence
0,Fluorescent Lamps,1.5,N,NULL,Lighting Resources,2023 Annual Tonnage Report ACE.json,Section C,0.9905,0.997,0.98,NULL,0.9935
1,Paint,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section C,0.995,NULL,NULL,NULL,NULL
2,"Textiles (clothing, shoes, leather, etc.)",NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section C,0.9934,NULL,NULL,NULL,NULL
3,Furniture,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section C,0.994,NULL,NULL,NULL,NULL
4,Books,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section C,0.995,NULL,NULL,NULL,NULL


### Section D

In [11]:
SectionDTotal = pd.DataFrame()

for filename in os.listdir(output_folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(output_folder_path, filename)
        try:
            with open(file_path, "r") as json_file:
                print(f"-------->Reading {filename}<---------")

                json_data = json.load(json_file)
                documents_list = json_data.get("documents", [])
                for document in documents_list:
                    SectionDT = pd.DataFrame()
                    
                    fields_dict = document.get("fields", {})
                    section_d_page1 = fields_dict.get("SectionD-Page1", {})
                    value_list_d1 = section_d_page1.get("value", [])
                    section_d_page2 = fields_dict.get("SectionD-Page2", {})
                    value_list_d2 = section_d_page2.get("value", [])

                    data_list_D1 = []
                    data_list_D2 = []

                    for item in value_list_d1:
                        field_data_D = {}
                        column1_value = item.get("value", {}).get("COLUMN1", {}).get("value", "")
                        field_data_D["Disposed/Recycled/Reused"] = column1_value

                        material_value = item.get("value", {}).get("Material", {}).get("value", "")
                        field_data_D["Material"] = material_value

                        tons_collected_value = item.get("value", {}).get("Tons Collected", {}).get("value", "")
                        field_data_D["Tons Collected"] = tons_collected_value

                        estimate_value = item.get("value", {}).get("Estimate? (Y/N)", {}).get("value", "")
                        field_data_D["Estimate? (Y/N)"] = estimate_value

                        facility_value = item.get("value", {}).get("Location(s) Where Disposed/Recycled/Reused", {}).get("value", "")
                        field_data_D["Location(s) Where Disposed/Recycled/Reused"] = facility_value

                        data_list_D1.append(field_data_D)
                    df_D1 = pd.DataFrame(data_list_D1)

                    for item in value_list_d2:
                        field_data_D2 = {}
                        column1_value = item.get("value", {}).get("COLUMN1", {}).get("value", "")
                        field_data_D2["Disposed/Recycled/Reused"] = column1_value

                        material_value = item.get("value", {}).get("Material", {}).get("value", "")
                        field_data_D2["Material"] = material_value

                        tons_collected_value = item.get("value", {}).get("Tons Collected", {}).get("value", "")
                        field_data_D2["Tons Collected"] = tons_collected_value

                        estimate_value = item.get("value", {}).get("Estimate? (Y/N)", {}).get("value", "")
                        field_data_D2["Estimate? (Y/N)"] = estimate_value

                        facility_value = item.get("value", {}).get("Location(s) Where Disposed/Recycled/Reused", {}).get("value", "")
                        field_data_D2["Location(s) Where Disposed/Recycled/Reused"] = facility_value

                        data_list_D2.append(field_data_D2)
                    df_D2 = pd.DataFrame(data_list_D2)

                    SectionD = pd.concat([df_D1, df_D2], axis=0)
                    SectionD.reset_index(drop=True, inplace=True)
                    SectionD.iloc[:, 1] = SectionD.iloc[:, 1].replace(',', '', regex=True)
                    SectionD = SectionD.map(lambda x: str(x).replace(':selected:', ''))
                    SectionD = SectionD.map(lambda x: str(x).replace('\n', ''))
                    SectionD = SectionD.map(lambda x: str(x).replace('nan', ''))

                    SectionD.iloc[:, 0] = SectionD.iloc[:, 0].replace("", None).ffill()

                    strings_to_check = ['MATERIAL','TONS COLLECTED', 'ESTIMATE? (Y/N)', 'LOCATION(S) WHERE DISPOSED/RECYCLED/REUSED']
                    SectionD = SectionD[~(SectionD.iloc[:, 1].str.upper().isin(strings_to_check))]# | SectionB.iloc[:, 2].str.upper().isin(strings_to_check))]
                    SectionD.iloc[:, 0] = SectionD.iloc[:, 0].replace("", None).ffill()

                    # Adding columns
                    SectionD['Filename'] = filename
                    SectionD['Section'] = 'Section D'

                    SectionDTotal = pd.concat([SectionDTotal, SectionD], ignore_index=True)

        except Exception as e:
            SkippedFiles.append(filename)
            print(f"Skipping file {filename} due to error: {str(e)}")
            continue

SectionDTotal = code_strings(SectionDTotal, 'Estimate? (Y/N)')
                  
# Removing Materials with Null values
SectionDTotal['Tons Collected'] = SectionDTotal['Tons Collected'].str.replace(r'[^\d. -]', '', regex=True)
SectionDTotal['Disposed/Recycled/Reused'] = SectionDTotal['Disposed/Recycled/Reused'].str.replace('Disposed Recycled', 'Recycled')

SectionDTotal = SectionDTotal[SectionDTotal['Tons Collected'].isin([0, "0", "o", pd.NA, 'N/A']) == False]
SectionDTotal = SectionDTotal.replace("", "NULL")
#SectionDTotal = SectionDTotal.dropna(subset=['Tons Collected'])

SectionDTotal.head(10)


-------->Reading 2023 Annual Tonnage Report ACE.json<---------
-------->Reading 2023 Annual Tonnage Report Chapel Valley.json<---------
-------->Reading 2023 Annual Tonnage Report Zero Waste Solutions.json<---------
-------->Reading 2023 Arlington Annual Tonnage Report-United Rentals.json<---------
-------->Reading 2023 Arlington Annual Tonnage Report.json<---------
-------->Reading 2023 Glenayr Apartments Tonnage Report.json<---------
-------->Reading 2023 Tonnage Report - Bartlett Tree.json<---------
-------->Reading ADS 2023 Tonnage Report.json<---------
-------->Reading Atlantic Biofuels Tonnage Report.json<---------
-------->Reading Capitol Services Tonnage Report 2024 Arlington County.json<---------
-------->Reading Compost Crew Annual Tonnage Report Arlingon.json<---------
-------->Reading Concrete Protection and Restoration 2023 Tonnage Report.json<---------
-------->Reading Darling Ingredients Tonnage Report.json<---------
-------->Reading Dittmar 2023 Annual Tonnage Report.js

,Disposed/Recycled/Reused,Material,Tons Collected,Estimate? (Y/N),Location(s) Where Disposed/Recycled/Reused,Filename,Section
0,Disposed,Office Trash,42.5,N,I-66 Landfill,2023 Annual Tonnage Report ACE.json,Section D
1,Recycled,C&D,153,N,Waste Management,2023 Annual Tonnage Report ACE.json,Section D
2,Reused,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section D
3,Recycled,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report Chapel Valley.json,Section D
4,Reused,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report Chapel Valley.json,Section D
5,Disposed,NULL,34.99,N,"Federal IPC- Recycle One, 2911 52nd Avenue, Hy...",2023 Annual Tonnage Report Zero Waste Solution...,Section D
6,Disposed,NULL,62.83,N,"Alexandria Waste Recovery Facility,3730 Greent...",2023 Annual Tonnage Report Zero Waste Solution...,Section D
7,Disposed,NULL,262.95,N,Federal IPC- Recycle One & Alexandria Waste Re...,2023 Annual Tonnage Report Zero Waste Solution...,Section D
8,Recycled,Wires,1.23,N,"Joseph Smith & Sons, 2001 Kenilworth Ave, Capi...",2023 Annual Tonnage Report Zero Waste Solution...,Section D
9,Recycled,Cardboard,146.67,N,"Federal IPC- Recycle One, 2911 52nd Avenue, Hy...",2023 Annual Tonnage Report Zero Waste Solution...,Section D


In [12]:
text_columns = [col for col in SectionDTotal.columns if col not in ['Filename', 'Section']]
SectionDTotal_Confidence = SectionDTotal.copy()
for col in text_columns:
    SectionDTotal_Confidence[col + '_confidence'] = SectionDTotal_Confidence.apply(lambda row: calculate_average_confidence(str(row[col]), row['Filename']), axis=1)

# Replacing zeros with hundred
SectionDTotal_Confidence.replace(0, "NULL", inplace=True)

SectionDTotal_Confidence.head()

,Disposed/Recycled/Reused,Material,Tons Collected,Estimate? (Y/N),Location(s) Where Disposed/Recycled/Reused,Filename,Section,Disposed/Recycled/Reused_confidence,Material_confidence,Tons Collected_confidence,Estimate? (Y/N)_confidence,Location(s) Where Disposed/Recycled/Reused_confidence
0,Disposed,Office Trash,42.5,N,I-66 Landfill,2023 Annual Tonnage Report ACE.json,Section D,0.995,0.9945,0.993,0.98,0.887
1,Recycled,C&D,153,N,Waste Management,2023 Annual Tonnage Report ACE.json,Section D,0.994,0.993,0.996,0.98,0.994
2,Reused,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section D,0.994,NULL,NULL,NULL,NULL
3,Recycled,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report Chapel Valley.json,Section D,0.994,NULL,NULL,NULL,NULL
4,Reused,NULL,NULL,NULL,NULL,2023 Annual Tonnage Report Chapel Valley.json,Section D,0.995,NULL,NULL,NULL,NULL


In [13]:
SectionETotal = pd.DataFrame()

# Iterate through each file in the folder
for filename in os.listdir(output_folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(output_folder_path, filename)
        try:
            with open(file_path, "r") as json_file:
                print(f"-------->Reading {filename}<---------")

                json_data = json.load(json_file)
                documents_list = json_data.get("documents", [])
                for document in documents_list:
                    SectionE = pd.DataFrame()

                    fields_dict = document.get("fields", {})
                    section_e = fields_dict.get("SectionE", {})
                    value_list = section_e.get("value", [])

                    data_list_E = []

                    for item in value_list:
                        field_data_E = {}
                        column1_value = item.get("value", {}).get("COLUMN1", {}).get("value", "")
                        field_data_E["Source Code"] = column1_value

                        # Assuming these keys are present in your JSON structure
                        tons_disposed_value = item.get("value", {}).get("Tons Disposed", {}).get("value", "")
                        field_data_E["Tons Disposed"] = tons_disposed_value

                        estimate_value = item.get("value", {}).get("Estimate? (Y/N)", {}).get("value", "")
                        field_data_E["Estimate? (Y/N)"] = estimate_value

                        facility_value = item.get("value", {}).get("Location(s) Where Disposed", {}).get("value", "")
                        field_data_E["Location(s) Where Disposed"] = facility_value

                        data_list_E.append(field_data_E)
                    
                    # Create DataFrame for SectionE data
                    SectionE = pd.DataFrame(data_list_E)

                    # Data cleaning and transformations
                    SectionE.reset_index(drop=True, inplace=True)
                    SectionE.iloc[:, 1] = SectionE.iloc[:, 1].replace(',', '', regex=True)
                    SectionE = SectionE.map(lambda x: str(x).replace(':selected:', ''))
                    SectionE = SectionE.map(lambda x: str(x).replace('\n', ''))
                    SectionE = SectionE.map(lambda x: str(x).replace('nan', ''))

                    SectionE.iloc[:, 0] = SectionE.iloc[:, 0].replace("", None).ffill()

                    strings_to_check = ['TONS DISPOSED', 'ESTIMATE? (Y/N)', 'LOCATION(S) WHERE DISPOSED']
                    SectionE = SectionE[~(SectionE.iloc[:, 1].str.upper().isin(strings_to_check))]# | SectionB.iloc[:, 2].str.upper().isin(strings_to_check))]
                    SectionE.iloc[:, 0] = SectionE.iloc[:, 0].replace("", None).ffill()

                    # Adding columns
                    SectionE['Filename'] = filename
                    SectionE['Section'] = 'Section E'

                    SectionETotal = pd.concat([SectionETotal, SectionE], ignore_index=True)

        except Exception as e:
            SkippedFiles.append(filename)
            print(f"Skipping file {filename} due to error: {str(e)}")
            continue

SectionETotal = code_strings(SectionETotal, 'Estimate? (Y/N)')
                  
# Removing Materials with Null values
SectionETotal['Tons Disposed'] = SectionETotal['Tons Disposed'].str.replace(r'[^\d. -]', '', regex=True)

SectionETotal = SectionETotal[SectionETotal['Tons Disposed'].isin([0, "0", "o", pd.NA, 'N/A']) == False]
SectionETotal = SectionETotal.replace("", "NULL")
#SectionETotal = SectionETotal.dropna(subset=['Tons Disposed'])

SectionETotal.head(10)


-------->Reading 2023 Annual Tonnage Report ACE.json<---------
-------->Reading 2023 Annual Tonnage Report Chapel Valley.json<---------
-------->Reading 2023 Annual Tonnage Report Zero Waste Solutions.json<---------
-------->Reading 2023 Arlington Annual Tonnage Report-United Rentals.json<---------
-------->Reading 2023 Arlington Annual Tonnage Report.json<---------
-------->Reading 2023 Glenayr Apartments Tonnage Report.json<---------
-------->Reading 2023 Tonnage Report - Bartlett Tree.json<---------
-------->Reading ADS 2023 Tonnage Report.json<---------
-------->Reading Atlantic Biofuels Tonnage Report.json<---------
-------->Reading Capitol Services Tonnage Report 2024 Arlington County.json<---------
-------->Reading Compost Crew Annual Tonnage Report Arlingon.json<---------
-------->Reading Concrete Protection and Restoration 2023 Tonnage Report.json<---------
-------->Reading Darling Ingredients Tonnage Report.json<---------
-------->Reading Dittmar 2023 Annual Tonnage Report.js

,Source Code,Tons Disposed,Estimate? (Y/N),Location(s) Where Disposed,Filename,Section
0,Residential (Single Family & Multifamily),NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section E
1,Commercial,60,Y,I-95 Landfill,2023 Annual Tonnage Report ACE.json,Section E
2,Industrial,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section E
3,Residential (Single Family & Multifamily),NULL,NULL,NULL,2023 Annual Tonnage Report Chapel Valley.json,Section E
4,Commercial,NULL,NULL,NULL,2023 Annual Tonnage Report Chapel Valley.json,Section E
5,Industrial,NULL,NULL,NULL,2023 Annual Tonnage Report Chapel Valley.json,Section E
6,Residential (Single Family & Multifamily),NULL,NULL,NULL,2023 Annual Tonnage Report Zero Waste Solution...,Section E
7,Commercial,4247.13,N,Covanta Alexandria/Arlington & Federal IPC- Re...,2023 Annual Tonnage Report Zero Waste Solution...,Section E
8,Industrial,NULL,NULL,NULL,2023 Annual Tonnage Report Zero Waste Solution...,Section E
9,Residential (Single Family & Multifamily),NULL,NULL,NULL,2023 Arlington Annual Tonnage Report-United Re...,Section E


In [14]:
text_columns = [col for col in SectionETotal.columns if col not in ['Filename', 'Section']]
SectionETotal_Confidence = SectionETotal.copy()
for col in text_columns:
    SectionETotal_Confidence[col + '_confidence'] = SectionETotal_Confidence.apply(lambda row: calculate_average_confidence(str(row[col]), row['Filename']), axis=1)

# Replacing zeros with hundred
SectionETotal_Confidence.replace(0, "NULL", inplace=True)

SectionETotal_Confidence.head(100)

,Source Code,Tons Disposed,Estimate? (Y/N),Location(s) Where Disposed,Filename,Section,Source Code_confidence,Tons Disposed_confidence,Estimate? (Y/N)_confidence,Location(s) Where Disposed_confidence
0,Residential (Single Family & Multifamily),NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section E,0.9866,NULL,NULL,NULL
1,Commercial,60,Y,I-95 Landfill,2023 Annual Tonnage Report ACE.json,Section E,0.9930,0.997,0.994,0.8705
2,Industrial,NULL,NULL,NULL,2023 Annual Tonnage Report ACE.json,Section E,0.9620,NULL,NULL,NULL
3,Residential (Single Family & Multifamily),NULL,NULL,NULL,2023 Annual Tonnage Report Chapel Valley.json,Section E,0.9762,NULL,NULL,NULL
4,Commercial,NULL,NULL,NULL,2023 Annual Tonnage Report Chapel Valley.json,Section E,0.9940,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...
98,Commercial,NULL,NULL,NULL,Safeguard Shredding Tonnage Report.json,Section E,0.9930,NULL,NULL,NULL
99,Industrial,NULL,NULL,NULL,Safeguard Shredding Tonnage Report.json,Section E,0.9920,NULL,NULL,NULL
100,Residential (Single Family & Multifamily),NULL,NULL,NULL,Safety kleen Systems Tonnage Report.json,Section E,0.9872,NULL,NULL,NULL
101,Commercial,NULL,NULL,NULL,Safety kleen Systems Tonnage Report.json,Section E,0.9930,NULL,NULL,NULL


In [15]:
data_list_A = []

for filename in os.listdir(output_folder_path):
    if filename.endswith(".json"):
        file = os.path.join(output_folder_path, filename)

        with open(file, "r") as json_file:
            print(f"-------->Reading {filename}<---------")
            json_data = json.load(json_file)

            documents_list = json_data.get("documents", [])

            # Section A
            for idx, document in enumerate(documents_list):
                field_data_A = {}
                for name, field in document.get("fields", {}).items():
                    field_value_A = field.get("valueString", "") if field.get("valueString", "") else field.get("content", "")
                    field_value_confidence = field.get("confidence", "")
                    field_data_A[name] = field_value_A
                    field_data_A[f"{name}_Confidence"] = field_value_confidence
                    field_data_A["Filename"] = filename

                data_list_A.append(field_data_A)

headers_A = [
    'Filename',
    'Company',
    'Company_Confidence',
    'Contact Person',
    'Contact Person_Confidence',
    'Title',
    'Title_Confidence',
    'Address',
    'Address_Confidence',
    'City',
    'City_Confidence',
    'State',
    'State_Confidence',
    'Zip Code',
    'Zip Code_Confidence',
    'Phone',
    'Phone_Confidence',
    'Fax',
    'Fax_Confidence',
    'Email',
    'Email_Confidence',
    'Paper Collection',
    'Paper Collection_Confidence',
    'Electronics Recycling',
    'Electronics Recycling_Confidence',
    'Secure Document Destruction',
    'Secure Document Destruction_Confidence',
    'Tire Recycling',
    'Tire Recycling_Confidence',
    'Solvents/Oil/Antifreeze Recycling',
    'Solvents/Oil/Antifreeze Recycling_Confidence',
    'Metals Recycling (Ferrous & Non-Ferrous)',
    'Metals Recycling (Ferrous & Non-Ferrous)_Confidence',
    'Battery Recycling',
    'Battery Recycling_Confidence',
    'Yard Maintenance/Landscaping',
    'Yard Maintenance/Landscaping_Confidence',
    'Textile, Shoes, and/or Clothing Recycling',
    'Textile, Shoes, and/or Clothing Recycling_Confidence',
    'Commercial Back-Hauling for Cardboard and Other',
    'Commercial Back-Hauling for Cardboard and Other_Confidence',
    'Materials Collection (grocery and other retail outlets)',
    'Materials Collection (grocery and other retail outlets)_Confidence',
    'Solid Waste Collection (trash and recycling)',
    'Solid Waste Collection (trash and recycling)_Confidence',
    'Construction and Demolition Debris Collection',
    'Construction and Demolition Debris Collection_Confidence',
    'Bulk Waste & Junk Collection',
    'Bulk Waste & Junk Collection_Confidence',
    'Material Recovery Facility (MRF)',
    'Material Recovery Facility (MRF)_Confidence',
    'Other',
    'Other_Confidence',
    'Other-Text',
    'Other-Text_Confidence',
    'Printed Signature',
    'Printed Signature_Confidence',
    'Signed Date',
    'Signed Date_Confidence',
    'Methodology',
    'Methodology_Confidence',
    'Comments',
    'Comments_Confidence'
]

df_A_Confidence = pd.DataFrame(data_list_A, columns=headers_A)
df_A_Confidence['Phone'] = df_A_Confidence['Phone'].str.replace(r'[^0-9]', '', regex=True)
df_A_Confidence['City'] = df_A_Confidence['City'].str.replace(r'[^a-zA-Z]', '', regex=True)
df_A_Confidence['Fax'] = df_A_Confidence['Fax'].str.replace(r'[^0-9]', '', regex=True)
df_A_Confidence.replace('', "NULL", inplace =True)
df_A_Confidence.replace(pd.NA, "NULL", inplace=True)

df_A_Confidence.head(10)

-------->Reading 2023 Annual Tonnage Report ACE.json<---------
-------->Reading 2023 Annual Tonnage Report Chapel Valley.json<---------
-------->Reading 2023 Annual Tonnage Report Zero Waste Solutions.json<---------
-------->Reading 2023 Arlington Annual Tonnage Report-United Rentals.json<---------
-------->Reading 2023 Arlington Annual Tonnage Report.json<---------
-------->Reading 2023 Glenayr Apartments Tonnage Report.json<---------
-------->Reading 2023 Tonnage Report - Bartlett Tree.json<---------
-------->Reading ADS 2023 Tonnage Report.json<---------
-------->Reading Atlantic Biofuels Tonnage Report.json<---------
-------->Reading Capitol Services Tonnage Report 2024 Arlington County.json<---------
-------->Reading Compost Crew Annual Tonnage Report Arlingon.json<---------
-------->Reading Concrete Protection and Restoration 2023 Tonnage Report.json<---------
-------->Reading Darling Ingredients Tonnage Report.json<---------
-------->Reading Dittmar 2023 Annual Tonnage Report.js

,Filename,Company,Company_Confidence,Contact Person,Contact Person_Confidence,Title,Title_Confidence,Address,Address_Confidence,City,City_Confidence,State,State_Confidence,Zip Code,Zip Code_Confidence,Phone,Phone_Confidence,Fax,Fax_Confidence,Email,Email_Confidence,Paper Collection,Paper Collection_Confidence,Electronics Recycling,Electronics Recycling_Confidence,Secure Document Destruction,Secure Document Destruction_Confidence,Tire Recycling,Tire Recycling_Confidence,Solvents/Oil/Antifreeze Recycling,Solvents/Oil/Antifreeze Recycling_Confidence,Metals Recycling (Ferrous & Non-Ferrous),Metals Recycling (Ferrous & Non-Ferrous)_Confidence,Battery Recycling,Battery Recycling_Confidence,Yard Maintenance/Landscaping,Yard Maintenance/Landscaping_Confidence,"Textile, Shoes, and/or Clothing Recycling","Textile, Shoes, and/or Clothing Recycling_Confidence",Commercial Back-Hauling for Cardboard and Other,Commercial Back-Hauling for Cardboard and Other_Confidence,Materials Collection (grocery and other retail outlets),Materials Collection (grocery and other retail outlets)_Confidence,Solid Waste Collection (trash and recycling),Solid Waste Collection (trash and recycling)_Confidence,Construction and Demolition Debris Collection,Construction and Demolition Debris Collection_Confidence,Bulk Waste & Junk Collection,Bulk Waste & Junk Collection_Confidence,Material Recovery Facility (MRF),Material Recovery Facility (MRF)_Confidence,Other,Other_Confidence,Other-Text,Other-Text_Confidence,Printed Signature,Printed Signature_Confidence,Signed Date,Signed Date_Confidence,Methodology,Methodology_Confidence,Comments,Comments_Confidence
0,2023 Annual Tonnage Report ACE.json,Ace Deconstruction LLC,0.937,Anchan Ladia,0.947,CFO,0.917,13804 Dawson Beach Road,0.947,Woodbridge,0.938,VA,0.940,22191,0.960,NULL,0.918,NULL,0.969,anchan@acedeconstruction.com,0.965,:selected:,0.983,:unselected:,0.983,:unselected:,0.983,:unselected:,0.985,:unselected:,0.983,:selected:,0.984,:selected:,0.984,:unselected:,0.983,:unselected:,0.983,:selected:,0.984,:unselected:,0.983,:selected:,0.982,:selected:,0.983,:unselected:,0.983,:unselected:,0.983,:unselected:,0.979,NULL,0.972,Anchan Ladia,0.948,1/09/2024,0.944,NULL,0.989,NULL,0.989
1,2023 Annual Tonnage Report Chapel Valley.json,Chapel Valley Landscape Company,0.948,JIll Hartkopf,0.964,Branch Manager,0.949,3041 Colvin Street,0.970,Alexandria,0.964,VA,0.948,22314,0.971,7035985514,0.942,NULL,0.968,jillh@chapelvalley.com,0.982,:unselected:,0.984,:unselected:,0.983,:unselected:,0.983,:unselected:,0.987,:unselected:,0.984,:unselected:,0.987,:unselected:,0.987,:unselected:,0.984,:unselected:,0.984,:unselected:,0.987,:unselected:,0.983,:selected:,0.982,:unselected:,0.983,:unselected:,0.986,:unselected:,0.984,:unselected:,0.981,NULL,0.972,Jill Hartkopf,0.965,01/05/2024,0.963,NULL,0.989,NULL,0.989
2,2023 Annual Tonnage Report Zero Waste Solution...,"ZERO WASTE SOLUTIONS, INC.",0.899,JESSICA SEXTON,0.940,ACCOUNTS PAYABLE SPECIALIST,0.877,"PO BOX 5097,",0.948,CONCORD,0.938,CA,0.949,94524,0.964,9252703339,0.918,NULL,0.965,ACCOUNTING @ZEROWASTESOLUTIONS.COM,0.937,:unselected:,0.983,:unselected:,0.983,:unselected:,0.983,:unselected:,0.986,:unselected:,0.983,:unselected:,0.986,:unselected:,0.986,:unselected:,0.983,:unselected:,0.983,:unselected:,0.986,:unselected:,0.983,:selected:,0.982,:selected:,0.983,:unselected:,0.984,:unselected:,0.983,:unselected:,0.980,NULL,0.972,JESSICA SEXTON,0.937,1/31/2024,0.937,NULL,0.988,NULL,0.989
3,2023 Arlington Annual Tonnage Report-United Re...,"United Rentals (North America), Inc .; DBA Rel...",0.939,Jeffrey Walker,0.957,Environmental Manager,0.948,"11825 Livingston Rd,",0.967,Manassas,0.962,VAS,0.955,20109,0.970,7036403901,0.948,NULL,0.968,environmental@ur.com; jwalker5@ur.com,0.981,:unselected:,0.983,:unselected:,0.983,:unselected:,0.983,:unselected:,0.986,:unselected:,0.983,:unselected:,0.986,:unselected:,0.986,:unselected:,0.983,:unselected:,0.983,:unselected:,0.985,:unselected:,0.983,:unselected:,0

In [16]:
excel_file_path = 'Annual_Tonnage_Sections_WithConfidenceV4.xlsx'
print('Skipped Files:', SkippedFiles)

with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    df_A.to_excel(writer, sheet_name='SectionA', index=False)
    df_A_Confidence.to_excel(writer, sheet_name='SectionA-Confidence', index=False)
    SectionBTotal.to_excel(writer, sheet_name='SectionB', index=False)
    SectionBTotal_Confidence.to_excel(writer, sheet_name='SectionB-Confidence', index=False)
    SectionCTotal.to_excel(writer, sheet_name='SectionC', index=False)
    SectionCTotal_Confidence.to_excel(writer, sheet_name='SectionC-Confidence', index=False)
    SectionDTotal.to_excel(writer, sheet_name='SectionD', index=False)
    SectionDTotal_Confidence.to_excel(writer, sheet_name='SectionD-Confidence', index=False)
    SectionETotal.to_excel(writer, sheet_name='SectionE', index=False)
    SectionETotal_Confidence.to_excel(writer, sheet_name='SectionE-Confidence', index=False)
    print('done!. Stored as Excel files in:', excel_file_path)

Skipped Files: ['Lancaster Landscapes Tonnage Report.json', 'Lancaster Landscapes Tonnage Report.json', 'Thompson Hauling Tonnage Report.json', 'Lancaster Landscapes Tonnage Report.json', 'Lancaster Landscapes Tonnage Report.json', 'Trash Away Tonnage Report.json']
done!. Stored as Excel files in: Annual_Tonnage_Sections_WithConfidenceV4.xlsx
